In [1]:
import os
%pwd

'c:\\Users\\latif\\OneDrive\\Belgeler\\GitHub\\Text_Summarization\\research'

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\latif\\OneDrive\\Belgeler\\GitHub\\Text_Summarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from text_summarization.constants import *
from text_summarization.utils.common import read_yaml, create_directories


In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
    
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
                root_dir=Path(config.root_dir),
                source_URL=config.source_URL,
                local_data_file=Path(config.local_data_file),
                unzip_dir=Path(config.unzip_dir)
            )

        return data_ingestion_config


    
        

In [6]:
import os
import urllib.request as request
import zipfile
from text_summarization.logging import logger
from text_summarization.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    

    def extract_zip_file(self):

        unzip_path = self.config.unzip_dir

        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-09-26 15:39:36,281: INFO: yaml file: config\config.yaml loaded successfully]
[2025-09-26 15:39:36,283: INFO: yaml file: params.yaml loaded successfully]
[2025-09-26 15:39:36,284: INFO: created directory at: artifacts]
[2025-09-26 15:39:36,286: INFO: created directory at: artifacts/data_ingestion]
[2025-09-26 15:39:37,178: INFO: artifacts\data_ingestion\data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: FF1E:18F472:5F3D:6ED9:68D68908
Accept-Ranges: bytes
Date: Fri, 26 Sep 2025 12:39:37 GMT
Via: 1.1 varnish
X-Served-By: cache-sof1510055-SOF
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1758890377.0529